In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import udf

In [2]:
spark = SparkSession \
    .builder \
    .appName("Market Analytics") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Load dataset

In [3]:
df = spark.read.format("csv").option("header", "true").load("dataset/tokopedia.csv")
df.show()

+--------------------+---------+---------+---------+--------------------+--------------------+
|          breadcrumb|item_sold|      pid|    price|               title|                 url|
+--------------------+---------+---------+---------+--------------------+--------------------+
|Dapur ;Bekal ;Cup...|        2|389668528|   70.000|Mayonnaise & Ketc...|https://www.tokop...|
|Rumah Tangga ;Keb...|        0|390053120|  160.000|BOLDe Super BROOM...|https://www.tokop...|
|Perawatan Hewan ;...|        0|390055891|1.100.000|Two Tone Brown (S...|https://www.tokop...|
|Fashion Muslim ;P...|        0|389610348|  325.000|MUKENA ALDIVA har...|https://www.tokop...|
|     Produk Lainnya |        1|389666117|  179.900|terpal kolam kota...|https://www.tokop...|
|Fashion Pria ;Aks...|        0|389609406|   58.000|TOPI TRUCKER CONS...|https://www.tokop...|
|Makanan & Minuman...|        0|389613659|  195.000|ARTE CHOCOLATE ( ...|https://www.tokop...|
|Perawatan Hewan ;...|        2|389671338|   10.00

In [4]:
df.dtypes

[('breadcrumb', 'string'),
 ('item_sold', 'string'),
 ('pid', 'string'),
 ('price', 'string'),
 ('title', 'string'),
 ('url', 'string')]

In [5]:
# some data have null in item_sold, pid, price, url
df = df.where(df['item_sold'].isNotNull())\
    .where(df['pid'].isNotNull())\
    .where(df['price'].isNotNull())\
    .where(df['title'].isNotNull())\
    .where(df['url'].isNotNull())

# remove wrong value
df = df.rdd.filter(lambda x: "http" not in x['item_sold'])\
    .filter(lambda x: "http" in x['url']).toDF()

In [34]:
@udf
def extract_seller(x):
    return x.split("/")[3]

@udf
def extract_seller_url(seller):
    return 'https://www.tokopedia.com/' + seller

@udf
def price(x):
    return int(x.replace(".", ""))

# extract more data
df = df.withColumn('seller', extract_seller(df.url))
df = df.withColumn('seller_url', extract_seller_url(df.seller))
df = df.withColumn('item_sold', df.item_sold.cast("int"))
df = df.withColumn('price', price(df.price))

df.show()

+--------------------+---------+---------+-------+--------------------+--------------------+----------------+--------------------+
|          breadcrumb|item_sold|      pid|  price|               title|                 url|          seller|          seller_url|
+--------------------+---------+---------+-------+--------------------+--------------------+----------------+--------------------+
|Dapur ;Bekal ;Cup...|        2|389668528|  70000|Mayonnaise & Ketc...|https://www.tokop...|      mkfamstore|https://www.tokop...|
|Rumah Tangga ;Keb...|        0|390053120| 160000|BOLDe Super BROOM...|https://www.tokop...|  bolde-official|https://www.tokop...|
|Perawatan Hewan ;...|        0|390055891|1100000|Two Tone Brown (S...|https://www.tokop...| vodkalattestore|https://www.tokop...|
|Fashion Muslim ;P...|        0|389610348| 325000|MUKENA ALDIVA har...|https://www.tokop...|     maisyahijab|https://www.tokop...|
|     Produk Lainnya |        1|389666117| 179900|terpal kolam kota...|https://www.

## Total Transaction (QTY)

In [35]:
quantities = df.agg(functions.sum('item_sold')).first()
quantities[0]

12298304

## Total Transaction (Rupiah)

In [36]:
transactions = df.agg(functions.sum('price')).first()
transactions[0]

862586607816.0

## Total Seller

In [39]:
df.select('seller').distinct().count()

90144

## Total Product

In [40]:
df.count()

1281288

## 10 Best Selling

In [46]:
df.groupBy('pid').agg(functions.sum('item_sold').alias('sale')).orderBy(functions.desc('sale')).show(10)

+---------+------+
|      pid|  sale|
+---------+------+
|315500698|105400|
|106556412| 97155|
|110892467| 88119|
|107184722| 77750|
|101040765| 73713|
|207550504| 72713|
|102077448| 65455|
|105978964| 64986|
|105110726| 56123|
|106156810| 49900|
+---------+------+
only showing top 10 rows



## 10 Best Seller

In [47]:
df.groupBy('seller').agg(functions.sum('item_sold').alias('sale')).orderBy(functions.desc('sale')).show(10)

+---------------+------+
|         seller|  sale|
+---------------+------+
|  rgaksesorishp|215670|
|  BAZAARFASHION|171702|
|      reawstore|142616|
|    tokosabilah|117575|
| mode-packaging|115712|
|      agenkurma|112732|
|       ikadella|105400|
|tokokainflanel1|104590|
|   platinumpack|100852|
|         190spt|100680|
+---------------+------+
only showing top 10 rows

